In [1]:
import acquire
import acquire_ry
import prepare
import scrape

import pandas as pd
import matplotlib as plt
import numpy as np
import time


In [2]:
prepdf = acquire_ry.get_github(cached=False)

In [3]:
prepdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214 entries, 0 to 213
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   language  214 non-null    object
 1   content   214 non-null    object
dtypes: object(2)
memory usage: 3.5+ KB


In [4]:
check_cache = acquire_ry.get_github(cached=True)

In [5]:
check_cache.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 214 entries, 0 to 213
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   language  214 non-null    object
 1   content   214 non-null    object
dtypes: object(2)
memory usage: 5.0+ KB


In [ ]:
first_ten = acquire_ry.get_github(cached=True)
first_ten

In [ ]:
base_url = 'https://github.com'
readme_url_list = []
for url in url_list:
    full_url = base_url + url
    readme_url_list.append(full_url)
    
readme_url_list

In [ ]:
soup = acquire.make_soup('https://github.com/emoncms/emoncms')

In [ ]:
content = soup.find('div', class_="Box-body px-5 pb-5")

In [ ]:
content

In [ ]:
language = soup.find('span', class_="text-gray-dark text-bold mr-1").text

In [ ]:
language

In [ ]:
df = acquire_ry.get_github(cached=True)
df

In [ ]:
print(range(1, 11)) 


In [ ]:
def github_urls():
    '''
    This function scrapes all of the evironmental urls from
    the github search page and returns a list of urls.
    '''
    # get the first 15 pages to allow for those that don't have readme or language
    pages = range(51, 100)
    urls = []
    
    for p in pages:
        
        # format string of the base url for the main github search page we are using to update with page number
        url = f'https://github.com/search?o=desc&p={p}&q=environmental&s=&type=Repositories'

        # Make request and soup object using helper
        soup = acquire_ry.make_soup(url)

        # Create a list of the anchor elements that hold the urls on this search page
        page_urls_list = soup.find_all('a', class_='v-align-middle')
        # for each url in the find all list get just the 'href' link
        page_urls = {link.get('href') for link in page_urls_list}
        # make a list of these urls
        page_urls = list(page_urls)
        # append the list from the page to the full list to return
        urls.append(page_urls)
        time.sleep(3)
        
    # flatten the urls list
    urls = [y for x in urls for y in x]
    return urls
# this produces a flattened list of lists

In [ ]:
urltest = github_urls()
len(urltest)

In [ ]:
urltest

In [ ]:
urltest2 = github_urls()
len(urltest2)

In [ ]:
urltest2

In [ ]:
flattened_list

In [ ]:
def get_github(url_list, cached=False):
    '''
    This function with default cached == False does a fresh scrape of github pages returned from
    search of 'environmental' and writes the returned df to a json file.
    cached == True returns a df read in from a json file.
    '''
    # option to read in a json file instead of scrape for df
    if cached == True:
        df = pd.read_json('readmes.json')
        
    # cached == False completes a fresh scrape for df    
    else:
#         # get url list
#         url_list = github_urls()

        # Set base_url that will be used in get request
        base_url = 'https://github.com'
        
        # List of full url needed to get readme info
        readme_url_list = []
        for url in url_list:
            full_url = base_url + url
            readme_url_list.append(full_url)
        
        # Create an empty list, readmes, to hold our dictionaries
        readmes = []

        for readme_url in readme_url_list:
            # Make request and soup object using helper
            soup = acquire_ry.make_soup(readme_url)

            if soup.find('article', class_="markdown-body entry-content container-lg") != None:            
                # Save the text in each readme to variable text
                content = soup.find('article', class_="markdown-body entry-content container-lg").text
            
            if soup.find('span', class_="text-gray-dark text-bold mr-1") != None:
            # Save the first language in each readme to variable text
                # NOTE: this is the majority language, not all of the languages used
                language = soup.find('span', class_="text-gray-dark text-bold mr-1").text

                # anything else useful on the page?

                # Create a dictionary holding the title and content for each blog
                readme = {'language': language, 'content': content}

                # Add each dictionary to the articles list of dictionaries
                readmes.append(readme)
            
        # convert our list of dictionaries to a df
        df = pd.DataFrame(readmes)

        # Write df to a json file for faster access
        df.to_json('readmes.json')

    
    return df

In [ ]:
testdf = get_github(urltest, cached=False)

In [ ]:
testdf.info()

In [ ]:
traindf = get_github(urltest2, cached=False)

In [ ]:
traindf.info()

In [ ]:
checkdf = pd.read_json('readmes.json')
checkdf.info()

In [ ]:
checkdf.head()